In [2]:
def parse_cfgfile(cfg_file):
    '''Parse configure file
    '''
    Aligned_Wig_files=''
    output_directory=''
    Annotated_3UTR_file=''
    Output_result_file=''
    Coverage_threshold = 1
    Num_threads = 1
    sequencing_depth_file = ''

    for line in open(cfg_file, 'r'):
        if line[0] == '\n' or line[0] == '#':
            comments = line;
        else:
            line = line.rstrip()
            command = line.split('=');
            if command[0] == 'Aligned_Wig_files':
                Aligned_Wig_files = command[1].split(',');
            if command[0] == 'Output_directory':
                output_directory = command[1]
                if output_directory[-1] != '/':
                    output_directory += '/'
            if command[0] == 'Annotated_3UTR':
                Annotated_3UTR_file = command[1]
            if command[0] == 'Output_result_file':
                Output_result_file = command[1]
            if command[0] == 'sequencing_depth_file':
                sequencing_depth_file = command[1]
            if command[0] == 'Num_Threads':
                Num_threads = int(command[1])
            if command[0] == 'Coverage_threshold':
                Coverage_threshold = int(command[1])


    if Aligned_Wig_files == '':
        print("No aligned BAM file found!", file=sys.stderr)
        exit(1)
    if output_directory=='':
        print("No output directory!", file=sys.stderr)
        exit(1)
    if Annotated_3UTR_file=='':
        print("No annotated 3' UTR file!", file=sys.stderr)
        exit(1)
    if Output_result_file=='':
        print("No result file name!", file=sys.stderr)
        exit(1)
    if sequencing_depth_file=='':
        print("No sequencing depth file!", file=sys.stderr)
        exit(1)

    return Aligned_Wig_files, output_directory, Annotated_3UTR_file, Output_result_file, sequencing_depth_file, Num_threads, Coverage_threshold


In [ ]:
(Group1_Tophat_aligned_file,
output_directory,
Annotated_3UTR_file,
Output_result_file,
sequencing_depth_file,
Num_threads,
Coverage_threshold)= parse_cfgfile(
'../DATA/DaPars_test_data_configure.txt')

In [10]:
def Load_Target_Wig_files_Multiple_threads_shared_dict_sampleid_key(All_Wig_files,UTR_Annotation_file, num_threads,curr_processing_chr):
    num_samples = len(All_Wig_files)
    UTR_events_dict = {}
    for line in open(UTR_Annotation_file, 'r'):
        fields = line.strip('\n').split('\t')
        curr_chr = fields[0]
        if curr_chr == curr_processing_chr:
            region_start = fields[1]
            region_end = fields[2]

            curr_strand = fields[-1]
            UTR_pos = "%s:%s-%s" %(curr_chr, region_start, region_end)
            end_shift = int(round(abs(int(region_start) - int(region_end)) * 0.2))
            if curr_strand == "+":
                region_end = str(int(region_end) - end_shift)
            else:
                region_start = str(int(region_start) + end_shift)
            region_start = int(region_start) + 1
            region_end = int(region_end) - 1
            if region_start + 50 < region_end:
                UTR_events_dict[fields[3]] = [fields[0],region_start,region_end,fields[-1],UTR_pos]

    Assigned_index = Assign_to_different_processor_balance(num_samples, num_threads)

    manager = multiprocessing.Manager() # create only 1 Manager
    All_samples_extracted_3UTR_coverage_dict = manager.dict() # create only 1 dict

    processes = []
    Final_assigned_threads_num = len(Assigned_index)
    for i in range(Final_assigned_threads_num):
        process = multiprocessing.Process(target=load_wig_funct_shared_dict_sampleid_key, args=(All_Wig_files, Assigned_index[i], UTR_events_dict,curr_processing_chr,All_samples_extracted_3UTR_coverage_dict))
        process.start()
        processes.append(process)

    for p in processes:
        p.join()

    return All_samples_extracted_3UTR_coverage_dict, UTR_events_dict



In [13]:

    (All_samples_Target_3UTR_coverages, UTR_events_dict 
     )= Load_Target_Wig_files_Multiple_threads_shared_dict_sampleid_key
    (All_Sample_files,
      Annotated_3UTR_file,
        Num_threads,
        curr_processing_chr)

TypeError: cannot unpack non-iterable function object